## Generalized Indices ground truth for SSZ-QL GI calculation
The target of this Notebook is to support the development of Generalized Indices calculation in the scope of Generalized Merkle Multiproofs

In [43]:
from eth2spec.altair.mainnet import get_generalized_index
from eth2spec.utils.ssz.ssz_typing import Container, List, Vector, Bytes32, uint64, byte, Bitlist, Bitvector, uint32, boolean
from typing import Union as PyUnion

In [44]:
class FixedNestedContainer(Container):
    Value1: uint64
    Value2: Vector[Bytes32, 32]  # Vector of Bytes32 with fixed length 32

class VariableNestedContainer(Container):
    Value1: uint64
    FieldListUint64: List[uint64, 100]
    NestedListField: List[List[byte, 100], 50]

class VariableOuterContainer(Container):
  Inner1: VariableNestedContainer
  Inner2: VariableNestedContainer

class VariableTestContainer(Container):
    LeadingField: Bytes32
    FieldListUint64: List[uint64, 2048]
    FieldListContainer: List[FixedNestedContainer, 128]
    FieldListBytes32: List[Bytes32, 100]
    Nested: VariableNestedContainer
    VariableContainerList: List[VariableOuterContainer, 10]
    BitlistField: Bitlist[2048]
    NestedListField: List[List[byte, 100], 50]
    TrailingField: Vector[byte, 56]

class FixedTestContainer(Container):
    field_uint32: uint32
    field_uint64: uint64
    field_bool: boolean
    field_bytes32: Bytes32
    nested: FixedNestedContainer
    vector_field: Vector[uint64, 24]
    two_dimension_bytes_field: Vector[Bytes32, 5]
    bitvector64_field: Bitvector[64]
    bitvector512_field: Bitvector[512]
    trailing_field: Vector[byte, 56]

In [45]:
# FixedNestedContainer
# Container field roots
element_paths = ['Value1', 'Value2']
for element in element_paths:
    gi = get_generalized_index(FixedNestedContainer, element)
    print(f"Generalized index for {element} is {gi}")

# Vector element roots (Value2 is a Vector[Bytes32, 32])
gi_v2_0 = get_generalized_index(FixedNestedContainer, 'Value2', 0)
print(f"Generalized index for Value2[0] is {gi_v2_0}")

gi_v2_31 = get_generalized_index(FixedNestedContainer, 'Value2', 31)
print(f"Generalized index for Value2[31] is {gi_v2_31}")

Generalized index for Value1 is 2
Generalized index for Value2 is 3
Generalized index for Value2[0] is 96
Generalized index for Value2[31] is 127


In [46]:
# VariableNestedContainer
# List length root (FieldListUint64 is a List[uint64, 100])
gi_list_len = get_generalized_index(VariableNestedContainer, 'FieldListUint64', '__len__')
print(f"Generalized index for len(FieldListUint64) is {gi_list_len}")

# Nested list examples (NestedListField is List[List[byte, 100], 50])
gi_outer_len = get_generalized_index(VariableNestedContainer, 'NestedListField', '__len__')
print(f"Generalized index for len(NestedListField) is {gi_outer_len}")

gi_inner0_len = get_generalized_index(VariableNestedContainer, 'NestedListField', 0, '__len__')
print(f"Generalized index for len(NestedListField[0]) is {gi_inner0_len}")

gi_inner_elem = get_generalized_index(VariableNestedContainer, 'NestedListField', 0, 5)
print(f"Generalized index for NestedListField[0][5] is {gi_inner_elem}")

Generalized index for len(FieldListUint64) is 11
Generalized index for len(NestedListField) is 13
Generalized index for len(NestedListField[0]) is 1537
Generalized index for NestedListField[0][5] is 6144


In [47]:
# VariableTestContainer
def _format_path(path):
    s = ""
    for seg in path:
        if isinstance(seg, str):
            if seg == "__len__":
                s += ".len()"
            else:
                s = f"{s}.{seg}" if s else seg
        else:  # int index
            s += f"[{seg}]"
    return s

# Paths to test (as tuples for consistency)
fields = [
    ("LeadingField",),
    ("FieldListUint64",),
    ("FieldListUint64", "__len__"),
    ("FieldListUint64", 0),
    ("FieldListUint64", 2047),
    ("Nested", "FieldListUint64", 10),
    ("BitlistField",),  # fixed: ensure 1-tuple
    ("BitlistField", 0),
    ("BitlistField", 1),
    ("BitlistField", "__len__"),  # handled specially below
    ("FieldListContainer", 0),
    ("FieldListContainer",),
    ("Nested",),
    ("FieldListBytes32",),
 ]

for path in fields:
    try:
        # Bitlist length is not supported by helper via '__len__'; compute as right-child of the bitlist root
        if len(path) == 2 and path[0] == "BitlistField" and path[1] == "__len__":
            root = get_generalized_index(VariableTestContainer, "BitlistField")
            gi = root * 2 + 1
        else:
            gi = get_generalized_index(VariableTestContainer, *path)
        print(f"GI for {_format_path(path)} is {gi}")
    except Exception as e:
        print(f"GI for {_format_path(path)} raised error: {e}")

GI for LeadingField is 16
GI for FieldListUint64 is 17
GI for FieldListUint64.len() is 35
GI for FieldListUint64[0] is 17408
GI for FieldListUint64[2047] is 17919
GI for Nested.FieldListUint64[10] is 5186
GI for BitlistField is 22
GI for BitlistField[0] is 352
GI for BitlistField[1] is 352
GI for BitlistField.len() is 45
GI for FieldListContainer[0] is 4608
GI for FieldListContainer is 18
GI for Nested is 20
GI for FieldListBytes32 is 19


In [48]:
# FixedTestContainer
fields = [
    'field_uint32',
    'field_uint64',
    'field_bool',
    'field_bytes32',
    'nested',
    'vector_field',
    'two_dimension_bytes_field',
    'bitvector64_field',
    'bitvector512_field',
    'trailing_field',
 ]
for name in fields:
    gi = get_generalized_index(FixedTestContainer, name)
    print(f"Generalized index for {name} is {gi}")

gi_bitvector64_0 = get_generalized_index(FixedTestContainer, 'bitvector64_field', 0)
print(f"Generalized index for bitvector64_field[0] is {gi_bitvector64_0}")
gi_bitvector64_63 = get_generalized_index(FixedTestContainer, 'bitvector64_field', 63)
print(f"Generalized index for bitvector64_field[63] is {gi_bitvector64_63}")

gi_bitvector512_0 = get_generalized_index(FixedTestContainer, 'bitvector512_field', 0)
print(f"Generalized index for bitvector512_field[0] is {gi_bitvector512_0}")
gi_bitvector512_511 = get_generalized_index(FixedTestContainer, 'bitvector512_field', 511)
print(f"Generalized index for bitvector512_field[511] is {gi_bitvector512_511}")

Generalized index for field_uint32 is 16
Generalized index for field_uint64 is 17
Generalized index for field_bool is 18
Generalized index for field_bytes32 is 19
Generalized index for nested is 20
Generalized index for vector_field is 21
Generalized index for two_dimension_bytes_field is 22
Generalized index for bitvector64_field is 23
Generalized index for bitvector512_field is 24
Generalized index for trailing_field is 25
Generalized index for bitvector64_field[0] is 23
Generalized index for bitvector64_field[63] is 23
Generalized index for bitvector512_field[0] is 48
Generalized index for bitvector512_field[511] is 49
